In [11]:
%load_ext autoreload
%autoreload 1

%aimport sheet
%aimport results_comparer
%aimport plot
%aimport settings_handler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
plot.display_image_settings()
#plot.show_final(3)

Output()

FloatSlider(value=0.95, continuous_update=False, description='Obstacle Threshold First row', layout=Layout(wid…

FloatSlider(value=0.9556699999999999, continuous_update=False, description='Obstacle Threshold Second row', la…

Checkbox(value=False, description='Sync sliders', indent=False)

Checkbox(value=True, description='Sync sliders', indent=False)

Button(description='Save Image', style=ButtonStyle())

AttributeError: 'types.SimpleNamespace' object has no attribute 'ignore'

In [ ]:
results_comparer.display_controls()

In [ ]:
sheet.initial_display()

In [ ]:
settings_handler.display_settings()